In [1]:
#Todos os imports necessários para aplicação.


import os
import librosa
import numpy as np
import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType

In [2]:
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import classpath_jars
from pyspark.sql.functions import create_map, struct
from pydub import AudioSegment
from soundscrape.soundscrape import process_soundcloud
import speech_recognition as sr

In [4]:
MODELED = '/home/rlfo/Documents/pessoal/bigdatamusic/modeled/'
RAW = '/home/rlfo/Documents/pessoal/bigdatamusic/raw/'

In [5]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("MUSIC SPARK")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark


In [12]:
r = sr.Recognizer()
to_append =''
for path in os.listdir('/home/rlfo/Documents/pessoal/bigdatamusic/landed/'):
    if('.wav' in path):
        audio_file = sr.AudioFile('/home/rlfo/Documents/pessoal/bigdatamusic/landed/'+path)
        with audio_file as source:
            audio = r.record(source)
            text = r.recognize_sphinx(audio)
            to_append+= f'{path};{text}'

with open(MODELED+'letters.csv','w') as letters:
    letters.write('name;text\n')
    letters.write(to_append)

In [ ]:
pipe_music = spark.sparkContext.parallelize(all_artist).foreach(download_audio)